# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.

First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.916429961087                   -0.69    4.9    117ms
  2   -7.921207358338       -2.32       -1.52    1.0   56.9ms
  3   -7.921404521096       -3.71       -2.49    1.4   64.2ms
  4   -7.921440855971       -4.44       -2.82    2.8   83.8ms
  5   -7.921441560485       -6.15       -2.96    1.0   59.7ms
  6   -7.921442005877       -6.35       -4.41    1.0   60.2ms
  7   -7.921442021801       -7.80       -4.61    2.9    155ms
  8   -7.921442022127       -9.49       -5.12    1.0   56.4ms
  9   -7.921442022142      -10.83       -5.96    1.2   58.7ms
 10   -7.921442022143      -11.90       -5.70    2.1   70.5ms
 11   -7.921442022144      -12.16       -6.75    1.0   55.8ms
 12   -7.921442022144      -13.73       -6.77    1.8   65.8ms
 13   -7.921442022144      -15.05       -7.09    1.0   56.4ms
 14   -7.921442022144      -14.75       -8.29    1.0    281ms


Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.39s  Non-interacting
   1        0       1        0.31     52.7   9.31s  
   2        0       2       -0.52     47.2   1.61s  
   3        0       3       -2.40     41.8   1.40s  
   4        0       4       -4.46     29.9   1.11s  
   5        0       5       -6.46     17.5   788ms  
   6        0       6       -8.92      4.0   441ms  
   7        1       1       -7.85     55.6   2.21s  Restart
   8        1       2       -9.07      5.5   507ms  
                                      72.8   2.12s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   1.94s  Non-interacting
   1        0       1        0.31     52.7   1.79s  
   2        0       2       -0.52     47.2   2.30s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.003498155370395767
 -0.0029774511719160538
 -0.0029774511704598384
  0.0043022306135756935
  0.004302230614478071
  0.004353201749370584